# Wstęp 
Preprocessing danych jest pierwszym i jednocześnie jednym z najważniejszych etapów, każdego projektu związanego z danologią (data science) oraz uczeniem maszynowym. Preprocessing skupia się na przetworzeniu posiadanych danych do postaci odpowiadającej wymaganiom danego zadania. Najczęściej polega to na usunięciu błędów takich jak brakujących lub niepotrzebnych danych (szumów). W tym dokumencie, chciałbym skupić się na procesie preprocessingu stosowanym w gałęzi uczenia maszynowego o nazwie **NLP** (ang. **Natural Language Processing**) czyli Przetwarzanie Języka Naturalnego, która zajmuje się badaniem możliwości AI w zadaniach dotyczących zagadnień językowych, takich jak rozumienie tekstu, tworzenie podsumowań, inteligentnych botów itp.
<br>
Do najczęściej wykorzystywanych kroków preprocessingu, należy zaliczyć:
- Usuwanie hiperlinków
- Usuwanie tagów HTML
- Stemming
- Lematyzacja
- Standaryzacja wielkości liter (tj. A→a)
- Usuwanie stopwordów
- Usuwanie częstych/rzadkich słów
- Usuwanie znaków interpunkcyjnych
- Usuwanie emotikon/emoji (szczególnie w przypadku danych z mediów społecznościowych)
- Przetwarzanie emotikon do słów
- Poprawianie błędów językowych

<br>
Każdy problem NLP wymaga własnego specyficznego podejścia przez co wykorzystywane procesy przetwarzania danych mogą się różnić w zależności od celów projektu.
<br>
Poniższy tekst w dużej mierze bazuje na gotowym kernelu, który można znaleźć pod linkiem:
<br>
https://www.kaggle.com/sudalairajkumar/getting-started-with-text-preprocessing

# Wymagane pakiety
Lista wymaganych do instalacji pakietów to:
- numpy | podstawowe operacje na obiektach
- pandas | przetwarzanie danych z pliku
- nltk | stemming/lematyzacja
- bs4 | operacje na plikach .html (usuwanie tagów)
- lxml | wymagane do działania BeautifulSoup
- pyspellchecker | biblioteka do sprawdzania poprawności zapisu
- morfeusz2 | biblioteka do analizy morfologicznej języka polskiego
- string
- collections
- os
- requests

<br>
W przypadku uruchomienia poniższego kodu, nastąpi pobranie (prawie) wszystkich wymaganych pakietów oraz plików. Może być wymagane ponowne uruchomienia Kernela w celu załadowania niektórych bibliotek.
<br>
Jedynym plikiem, którego pobieranie nie jest zautomatyzowane jest plik z danymi.
<br>
Nalezy pobrac plik .csv spod linku https://www.kaggle.com/kazanova/sentiment140
<br>
Domyślnie ścieżka do ładowanego pliku wymaga umieszczenia go w folderze w którym znajduje się notatnik.

In [3]:
%conda install numpy
%conda install pandas
%conda install nltk
%conda install bs4
%conda install lxml
%conda install pyspellchecker

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [4]:
import os, requests
'''
download
Funckja download jest wykorzystywana w celu pobierania wymaganych plików
źródło: https://stackoverflow.com/questions/22676/how-do-i-download-a-file-over-http-using-python

Parametry:
    url: adres html pod którym znajduje się porządany plik
'''
def download(url: str):
    get_response = requests.get(url,stream=True)
    file_name  = url.split("/")[-1]
    with open(file_name, 'wb') as f:
        for chunk in get_response.iter_content(chunk_size=1024):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
download("http://download.sgjp.pl/morfeusz/20200906/Windows/64/morfeusz2-1.9.16-py3.7-win-amd64.egg")
!easy_install morfeusz2-1.9.16-py3.7-win-amd64.egg

Processing morfeusz2-1.9.16-py3.7-win-amd64.egg
removing 'c:\users\senuk\anaconda3\envs\preprocessing\lib\site-packages\morfeusz2-1.9.16-py3.7-win-amd64.egg' (and everything under it)
creating c:\users\senuk\anaconda3\envs\preprocessing\lib\site-packages\morfeusz2-1.9.16-py3.7-win-amd64.egg
Extracting morfeusz2-1.9.16-py3.7-win-amd64.egg to c:\users\senuk\anaconda3\envs\preprocessing\lib\site-packages
morfeusz2 1.9.16 is already the active version in easy-install.pth

Installed c:\users\senuk\anaconda3\envs\preprocessing\lib\site-packages\morfeusz2-1.9.16-py3.7-win-amd64.egg
Processing dependencies for morfeusz2==1.9.16
Finished processing dependencies for morfeusz2==1.9.16


In [5]:
import numpy as np
import pandas as pd
import re
import nltk
import string
import morfeusz2
pd.options.mode.chained_assignment = None

# Opis danych
Do demonstracji algorytmów preprocessingu wykorzystano zestaw danych *sentiment140*, zawierający 1,6mln tweetów.
<br>
Dane zawierają następujące pola:
- target: typ zdania (0 = negatywne, 2 = neutralne, 4 = pozytywne)
- ids: ID tweeta (2087)
- date: data tweeta (Sat May 16 23:58:44 UTC 2009)
- flag: typ zapytania, jeśli brak to NO_QUERY.
- user: nazwa użytkownika, który napisał tweet (robotickilldozr)
- text: zawartość tweeta (Lyx is cool)

<br>
W dalszych etapie wykorzystamy pierwsze 10000 tweetów.

In [6]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", nrows=10000)
data.columns = ['target','id','date','flag','user','text']
text_data = data[["text"]]
text_data["text"] = text_data["text"].astype(str)
data.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


# Lower casing 
Technika *lower casing* polega na formatowaniu tekstu do postaci małych liter. Jest to szczególnie istotne w zastosowaniach w których, ważnym jest, aby te same słowa zapisane w różny sposób były traktowane jako jeden wyraz.
<br>
Na przykład „WORD”, „Word” i „word” zostaną sformatowane do postaci „word”.
<br>
Użyteczność *lower casingu* ujawnia się w zastosowaniach powiązanych z częstością występowania wyrazów, poprzez zmniejszenie ilości duplikatów. Przykładem takiego zastosowania może być np. TFDIF (pol. ważenie częstością termów), która oblicza wagi słów w oparciu o liczbę ich wystąpień.
<br>
Należy jednak pamiętać, że wykorzystanie tej techniki nie jest pomocne we wszystkich zastosowaniach związanych z NLP, gdyż w niektórych wypadkach wielkość liter, może zawierać przydatne informacje o wyrażanych emocjach, czy po prostu częściach mowy/zdania.
<br>
Wszystkie najważniejsze biblioteki zawierają gotowe funkcje konwertujące tekst do postaci małych liter, bądź po prostu wykonują tą czynność same z siebie, jako element algorytmu. 
Przykładowe biblioteki i funkcje to:
- NLTK
- TensorFlow/Keras Tokenizer 
- sklearn.feature_extraction.text.CountVectorizer

<br>
W tym przypadku wykorzystano funkcje wbudowana w bibliotece *pandas*.

In [7]:
text_data["text_lower_case"] = text_data["text"].str.lower()
text_data.head()

,text,text_lower_case
0,is upset that he can't update his Facebook by ...,is upset that he can't update his facebook by ...
1,@Kenichan I dived many times for the ball. Man...,@kenichan i dived many times for the ball. man...
2,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
3,"@nationwideclass no, it's not behaving at all....","@nationwideclass no, it's not behaving at all...."
4,@Kwesidei not the whole crew,@kwesidei not the whole crew


# Usuwanie znaków interpunkcyjnych
Technika ta polega na usuwaniu znaków interpunkcyjnych występujących w tekście.
<br>
Pozwala  na pozbycie się zbędnych szumów, które zazwyczaj nie niosą żadnych istotnych informacji.
<br>
Ponadto usunięcie znaków interpunkcyjnych pozwala traktować np. „Hey!” i „Hey” w ten sam sposób.
<br>
Python zawiera wbudowany string zawierający wszystkie znaki interpunkcyjne, pod poleceniem string.punctuation.
<br>
Wykorzystana funkcja została znaleziona pod linkiem:
<br>
https://www.pythondaddy.com/python/how-to-remove-punctuation-from-a-dataframe-in-pandas-and-python/


In [8]:
PUNCTUATION = string.punctuation
text_data.drop(["text_lower_case"], axis=1, inplace=True)
def remove_punctuation(x: str) -> str:
    try:
        x = x.str.replace('[^\w\s]','')
    except:
        pass
    return x
text_data["text_no_punct"] = text_data.apply(remove_punctuation)
text_data.head()

,text,text_no_punct
0,is upset that he can't update his Facebook by ...,is upset that he cant update his Facebook by t...
1,@Kenichan I dived many times for the ball. Man...,Kenichan I dived many times for the ball Manag...
2,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
3,"@nationwideclass no, it's not behaving at all....",nationwideclass no its not behaving at all im ...
4,@Kwesidei not the whole crew,Kwesidei not the whole crew


# Usuwanie stop-słów 
Stop lista to lista zawierająca słowa nie wpływające na identyfikację dokumentu. Tego typu słowami są np. spójniki oraz słowa popularne. Istnieją gotowe stop listy dla różnych języków, w tym polskiego, która zawiera 350 słów.
<br>
Najważniejsze biblioteki posiadają wbudowane funkcje służące do filtracji słów ze stop list. Takimi bibliotekami są:
- NLTK
- TensorFlow

In [9]:
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english'))
'''
remove_stopwords
Funckja remove_stopwords wykorzystuje stop listę z biblioteki nltk.
Pobiera tekst w postaci stringa, który następnie jest dzielony na pojedyncze wyrazy.
Jeśli wyraz wyraz nie znajduje się na stop liście to jest on dodawany tuple (krotki).
Funkcja na koniec łączy wszystkie wyrazy z krotki ze spacją jako separatorem.

Parametry:
    text: tekst z którego mają zostać usunięte stopwordy
'''
def remove_stopwords(text: str) -> str:
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

text_data["text_no_stop"] = text_data["text_no_punct"].apply(lambda text: remove_stopwords(text))
text_data.head()

,text,text_no_punct,text_no_stop
0,is upset that he can't update his Facebook by ...,is upset that he cant update his Facebook by t...,upset cant update Facebook texting might cry r...
1,@Kenichan I dived many times for the ball. Man...,Kenichan I dived many times for the ball Manag...,Kenichan I dived many times ball Managed save ...
2,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,whole body feels itchy like fire
3,"@nationwideclass no, it's not behaving at all....",nationwideclass no its not behaving at all im ...,nationwideclass behaving im mad I cant see
4,@Kwesidei not the whole crew,Kwesidei not the whole crew,Kwesidei whole crew


# Usuwanie częstych/rzadkich słów
Zadaniem tego procesu, podobnie do usuwania stop-słów, jest odfiltrowanie zbędnych słów ze zbioru danych, które nie niosą żadnych istotnych dla zadania informacji.
<br>
Operację usuwania częstych słów można wykonać przy użyciu modułu *Counter* z biblioteki *collections*.
<br>
Należy zliczyć słowa występujące w dokumencie, a następnie wybrać np. 10 najczęstszych i je usunąć.
<br>
Taki sam proces można wykonać na najrzadszych słowach w korpusie.

In [10]:
from collections import Counter

#Zliczanie wszystkich słów w zbiorze
counter = Counter()
for text in text_data["text_no_stop"].values:
    for word in text.split():
        counter[word] += 1
counter.most_common(10)

n_rare_words = 10
# Tworzenie zbiorów 10 najczęstszych i najrzadszych wyrazów
FREQWORDS = set([w for (w, wc) in counter.most_common(10)])
RAREWORDS = set([w for (w, wc) in counter.most_common()[:-n_rare_words-1:-1]])

'''
remove_common_words
Funkcja remove_common_words pobiera tekst w postaci stringa, 
który następnie jest dzielony na pojedyncze wyrazy.
Jeśli wyraz wyraz nie znajduje się na liście częstych wyrazów to jest on dodawany tuple (krotki).
Funkcja na koniec łączy wszystkie wyrazy z krotki ze spacją jako separatorem.

Parametry:
    text: tekst z którego mają zostać usunięte częste wyrazy
'''
def remove_common_words(text: str) -> str:
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

'''
remove_uncommon_words
Funkcja remove_uncommon_words pobiera tekst w postaci stringa, 
który następnie jest dzielony na pojedyncze wyrazy.
Jeśli wyraz wyraz nie znajduje się na liście najrzadszych wyrazów to jest on dodawany tuple (krotki).
Funkcja na koniec łączy wszystkie wyrazy z krotki ze spacją jako separatorem.

Parametry:
    text: tekst z którego mają zostać usunięte najrzadsze wyrazy
'''
def remove_uncommon_words(text: str) -> str:
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

text_data["text_no_stopfreq"] = text_data["text_no_stop"].apply(lambda text: remove_common_words(text))
text_data["text_no_stoprare"] = text_data["text_no_stopfreq"].apply(lambda text: remove_uncommon_words(text))

text_data.drop(["text_no_punct", "text_no_stop"], axis=1, inplace=True)
text_data.head()

,text,text_no_stopfreq,text_no_stoprare
0,is upset that he can't update his Facebook by ...,upset update Facebook texting might cry result...,upset update Facebook texting might cry result...
1,@Kenichan I dived many times for the ball. Man...,Kenichan dived many times ball Managed save 50...,Kenichan dived many times ball Managed save 50...
2,my whole body feels itchy and like its on fire,whole body feels itchy fire,whole body feels itchy fire
3,"@nationwideclass no, it's not behaving at all....",nationwideclass behaving im mad see,nationwideclass behaving im mad see
4,@Kwesidei not the whole crew,Kwesidei whole crew,Kwesidei whole crew


# Stemming
Technika stemmingu polega na usunięciu końcówki fleksyjnej słowa w celu pozostawienia jedynie tematu wyrazu (do rdzenia – ang. stem, stąd stemming).
<br>
Na przykład jeśli w korpusie znajdują się dwa słowa wywodzące się z jednego wyrazu np. „talk” i „talking” to algorytm stemmingu usunie końcówkę -ing i obydwa będą miały formę „talk”.
<br>
Jednym z najpopularniejszych algorytmów stemmingu jest algorytm Portera. Popularne biblioteki takie jak NLTK, scikit-learn czy TensorFlow nie zawierają algorytmów działających dla języka polskiego.
Opis algorytmu Portera można znaleźć pod linkiem: 
<br>
http://snowball.tartarus.org/algorithms/porter/stemmer.html

In [11]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

# Usuwanie niepotrzebnych kolumn
text_data.drop(["text_no_stopfreq", "text_no_stoprare"], axis=1, inplace=True) 

porter_stemmer = PorterStemmer()
snowball_stemmer = SnowballStemmer("english")
'''
stem_words
Funkcja stem_words pobiera tekst w postaci stringa, który następnie jest dzielony na pojedyncze wyrazy.
Każdy kolejny wyraz jest poddawany procesowi stemmmingu i zwracany do tuple (krotki).
Funkcja na koniec łączy wszystkie wyrazy z krotki ze spacją jako separatorem.

Parametry:
    stemmer: obiekt zawierający wybrany algorytm stemmingu
    text: tekst z którego mają zostać usunięte częste wyrazy
'''
def stem_words(stemmer: object, text: str) -> str:
    return " ".join([stemmer.stem(word) for word in text.split()])

text_data["text_stemmed"] = text_data["text"].apply(lambda text: stem_words(porter_stemmer,text))
text_data["text_stemmed_snow"] = text_data["text"].apply(lambda text: stem_words(snowball_stemmer,text))
text_data.head()

,text,text_stemmed,text_stemmed_snow
0,is upset that he can't update his Facebook by ...,is upset that he can't updat hi facebook by te...,is upset that he can't updat his facebook by t...
1,@Kenichan I dived many times for the ball. Man...,@kenichan I dive mani time for the ball. manag...,@kenichan i dive mani time for the ball. manag...
2,my whole body feels itchy and like its on fire,my whole bodi feel itchi and like it on fire,my whole bodi feel itchi and like it on fire
3,"@nationwideclass no, it's not behaving at all....","@nationwideclass no, it' not behav at all. i'm...","@nationwideclass no, it not behav at all. i'm ..."
4,@Kwesidei not the whole crew,@kwesidei not the whole crew,@kwesidei not the whole crew


# Lematyzacja
Technika o podobnym założeniu do stemmingu, jednak jej celem nie jest sprowadzenie słowa do postaci bazowej w sposób algorytmiczny, ale przez wykorzystanie gotowych słowników (słowo-sieci).
<br>
W ten sposób lematyzacja gwarantuje, że otrzymane słowo istnieje i jest poprawne gramatycznie (proces stemmingu, może uciąć końcówkę, bez której otrzymane wyraz nie jest prawdziwy słowem np. „wolves” → „wolv”).
<br>
Możliwym jest również, aby transformować słowa do różnych postaci np. rzeczownika, czasownika itd.
Biblioteka *NLTK* zawiera WordNet dla m.in. języka angielskiego, jednak w przypadku języka polskiego, wymagane jest korzystanie zewnętrznego oprogramowania jak np. *Morfeusz2* lub zestawu narzędzi *LemmaPL*.

In [16]:
from nltk.stem import WordNetLemmatizer

# Usuwanie niepotrzebnej kolumny
text_data.drop(["text_stemmed_snow"], axis=1, inplace=True) 

'''
lemmatize_words
Funkcja lemmatize_words pobiera tekst w postaci stringa, 
który następnie jest dzielony na pojedyncze wyrazy.
Każdy kolejny wyraz jest poddawany procesowi lematyzacji i zwracany do tuple (krotki).
Funkcja na koniec łączy wszystkie wyrazy z krotki ze spacją jako separatorem.

Parametry:
    lemmatizer: obiekt zawierający wybrany algorytm lematyzacji
    text: tekst z którego mają zostać usunięte częste wyrazy
'''
lemmatizer = WordNetLemmatizer()
def lemmatize_words(lemmatizer: object, text: str) -> str:
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

text_data["text_lemmatized"] = text_data["text"].apply(lambda text: lemmatize_words(lemmatizer, text))
text_data.head()

,text,text_stemmed,text_lemmatized
0,is upset that he can't update his Facebook by ...,is upset that he can't updat hi facebook by te...,is upset that he can't update his Facebook by ...
1,@Kenichan I dived many times for the ball. Man...,@kenichan I dive mani time for the ball. manag...,@Kenichan I dived many time for the ball. Mana...
2,my whole body feels itchy and like its on fire,my whole bodi feel itchi and like it on fire,my whole body feel itchy and like it on fire
3,"@nationwideclass no, it's not behaving at all....","@nationwideclass no, it' not behav at all. i'm...","@nationwideclass no, it's not behaving at all...."
4,@Kwesidei not the whole crew,@kwesidei not the whole crew,@Kwesidei not the whole crew


In [17]:
example1 = lemmatizer.lemmatize("cooking")
example2 = lemmatizer.lemmatize("cooking","v")
print("Lematyzacja bez określenia POS: " + example1)
print("Lematyzacja przy określeniu POS: " + example2)

Lematyzacja bez określenia POS: cooking
Lematyzacja przy określeniu POS: cook


In [18]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
#Mapowanie części zdania
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
'''
lemmatize_words
Ulepszona funkcja lemmatize_words pobiera tekst w postaci stringa, 
który następnie jest dzielony na pojedyncze wyrazy.
Tworzony jest obiekt zawierający pary (wyraz, część zdania), każdego słowa w zadanym tekście.
Metoda lemmatize przyjmuje jako parametry wejściowe zadane słowo oraz typ części zdania.
Funkcja na koniec łączy wszystkie wyrazy z krotki ze spacją jako separatorem.

Parametry:
    lemmatizer: obiekt zawierający wybrany algorytm lematyzacji
    text: tekst z którego mają zostać usunięte częste wyrazy
'''
def lemmatize_words(lemmatizer: object, text: str) -> str:
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, 
                                                                         pos in pos_tagged_text])

text_data["text_lemmatized"] = text_data["text"].apply(lambda text: lemmatize_words(lemmatizer, text))
text_data.head()

,text,text_stemmed,text_lemmatized
0,is upset that he can't update his Facebook by ...,is upset that he can't updat hi facebook by te...,be upset that he can't update his Facebook by ...
1,@Kenichan I dived many times for the ball. Man...,@kenichan I dive mani time for the ball. manag...,@Kenichan I dive many time for the ball. Manag...
2,my whole body feels itchy and like its on fire,my whole bodi feel itchi and like it on fire,my whole body feel itchy and like it on fire
3,"@nationwideclass no, it's not behaving at all....","@nationwideclass no, it' not behav at all. i'm...","@nationwideclass no, it's not behave at all. i..."
4,@Kwesidei not the whole crew,@kwesidei not the whole crew,@Kwesidei not the whole crew


# Usuwanie emoji/emotikon
Proces usuwania emotikon wykorzystuje gotową funkcję, którą można znaleźć pod linkiem:
<br>
https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
<br>
Na tym etapie wykorzystwany jest język wyrażeń regularnych, który pozwala na ogólny opis parametrów wzorca zdania. Taki wzorzec może zostać wykorzystany do wyszukiwania zdań o takiej samej postaci.
W tym przypadku tworzony jest wzorzec emoji/emotikon, który pozwala na szybkie wyszukanie wszystkich wystąpień w danym tekście i zastąpienie ich pustym polem.
<br>
Dokładniejszy opis języka wyrażeń regularnych można znaleźć pod linkiem:
<br>
https://docs.python.org/3/library/re.html

In [19]:
'''
remove_emoji
Funckja remove_emoji wykorzystuje język wyrażeń regularnych w celu stworzenia szablonu wszystkich
możliwych wystąpień emoji.
Na podstawie szablonu funkcja zastępuje wszystkie wystąpienia emoji pustym polem.

Parametry:
    text: tekst z którego mają zostać usunięte wszystkie emoji
'''
def remove_emoji(text: str) -> str:
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emotikony
                           u"\U0001F300-\U0001F5FF"  # symbole i piktogramy
                           u"\U0001F680-\U0001F6FF"  # symbole transportu i map
                           u"\U0001F1E0-\U0001F1FF"  # flagi (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

remove_emoji("I like cats 🐱")

'I like cats '

In [20]:
# Pobiernaie zbioru emotikon
download("https://raw.githubusercontent.com/NeelShah18/emot/master/emot/emo_unicode.py")
from emo_unicode import EMOTICONS, UNICODE_EMO

'''
remove_emoticons
Funckja remove_emoticons wykorzystuje język wyrażeń regularnych w celu stworzenia szablonu wszystkich
możliwych wystąpień emotikon. Szablon jest tworzony na postawie gotowego zbioru emotikon.
Na podstawie szablonu funkcja zastępuje wszystkie wystąpienia emotikon pustym polem.

Parametry:
    text: tekst z którego mają zostać usunięte wszystkie emotikon
'''
def remove_emoticons(text: str) -> str:
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

remove_emoticons("I am very happy :-)")

'I am very happy '

# Konwersja emotikon do słów
W przypadku analizy tekstów z platform społecznościowych emoji oraz emotikony, mogą wnosić bardzo dużo przydatnych informacji np. o zabarwieniu zdania (czy jest negatywne, pozytywne, sarkastyczne itp.).
<br>
W takiej sytuacji usuwanie ich nie jest zalecane, jednak wymagane jest przetworzenie ich do postaci słownej, aby mogły zostać wykorzystane w procesie uczenia modelu/modelowania.
<br>
Istnieją gotowe listy słów korespondujących z emoji/emotikonami.

In [21]:
'''
convert_emoticons
Funckja convert_emoticons wykorzystuje język wyrażeń regularnych w celu 
zastąpienia wszystkich wystąpień emotikon na opis słowny.

Parametry:
    text: tekst w którym wszystkie emotikony mają zostać zastąpione opisem słownym
'''
def convert_emoticons(text: str) -> str:
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
    return text

text = "Hello ;-)"
convert_emoticons(text)

text = "I am sad :'("
convert_emoticons(text)

'I am sad Crying'

# Usuwanie URLów i tagów HTML
Kolejnym krokiem, który może być konieczny w procesie preprocessingu, może być usuanie URLów i tagów HTML, w przypadku gdy pracujemy na danych które zostały np. zescrapowane ze strony internetowej.
<br>
Usuwanie urlów może zostać wykonane poprzez stworzenie np. prostego wzorca URLu.
<br>
Usuwanie tagów może zostać przeprowadzone ręcznie poprzez stworzenie wzorca tagów lub przy wykorzystaniu biblioteki BeautifulSoup4, która posiada gotową funkcję pozwalająca przetworzyć zescrapowany html do postaci czystego tekstu.


In [28]:
'''
remove_urls
Funckja remove_urls wykorzystuje język wyrażeń regularnych w celu stworzenia szablonu
możliwych hiperlinków.
Na podstawie szablonu wszystkie linki występujące w zadanym tekście są zastępowane pustym polem.

Parametry:
    text: tekst z którego mają zostać wykasowane wszystkie linki
'''
def remove_urls(text: str) -> str:
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

text = "Snippets of the album can be listened on distributor’s server under this link. www.alternation.eu"
remove_urls(text)

'Snippets of the album can be listened on distributor’s server under this link. '

In [29]:
from bs4 import BeautifulSoup

def remove_html(text: str) -> str:
    """Funckja wykorzystuje moduł BeautifulSoup do usunięcia wszystkich tagów HTML z zadanego tekstu"""
    return BeautifulSoup(text, "lxml").text

text = """<div>
<h1>TEST</h1>
<p>ANOTHER TEST</p>
<a href="www.google.com"> GOOGLE</a>
</div>
"""

print(remove_html(text))


TEST
ANOTHER TEST
 GOOGLE




# Poprawianie błędów językowych
Technika ta wykorzystuje gotowe algorytmy do wykrywania błędów w zapisie, w celu ich korekty.
<br>
Wykrywanie i korekta błędów zapisu jest bardzo ważna w przypadku analizy tekstów z portali społecznościowych. Błędnie zapisane słowa mogą zawierać wiele przydatnych informacji, które nie zostaną poprawnie zainterpretowane.
<br>
Algorytm wykrywania błędów można znaleźć w bibliotece *pyspellchecker*.

In [33]:
from spellchecker import SpellChecker

'''
correct_spellings
Funckja correct_spellings wykorzystuje moduł SpellChecker do poprawiania błędów w zapisie słów.
Pierwszym krokiem jest sprawdzenie poprawności wszystkich słów w zadanym tekście.
Błędnie zapisane słowa są zapisywane w osobnej tablicy.
Następnie wszystkie słowa, które wymagają poprawy są poprawiane i zapisywane w gotowej tablicy.
Ostatnim krokiem jest połączenie tablicy do postaci stringa.

Parametry:
    text: tekst, który ma zostać zbadany pod kątem poprawności zapisu oraz w razie potrzeby poprawiony
'''
spell = SpellChecker()
def correct_spellings(text: str) -> str:
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)
        
text = "speling mistaks"
correct_spellings(text)

'spelling mistake'

# Analiza morfologiczna dla języka polskiego
W przypadku języka polskiego występują jedynie nieliczne (open-source) narzędzia do NLP.
<br>
Jednym z dostępnych narzędzi jest darmowy Morfeusz2.
<br>
Link do projektu Morfeusz:
<br>
http://morfeusz.sgjp.pl/
<br>
Link do dokumentacji:
<br>
http://download.sgjp.pl/morfeusz/Morfeusz2.pdf
<br>
Opis działania Morfeusza (z dokumentacji znajdującej się na oficjalnej stronie projektu):
<br>
"Dwie najważniejsze metody klasy Morfeusz to analyse oraz generate.
Pierwsza z nich zwraca graf analizy morfoskładniowej dla podanego napisu
w postaci listy trójek uporządkowanych reprezentujących pojedyncze interpretacje poszczególnych segmentów (czyli krawędzie w grafie analizy).
<br>
Każda trójka składa się z indeksów węzła początkowego i końcowego danej krawędzi
oraz z interpretacji morfoskładniowej, stanowiącej etykietę krawędzi.
<br>
Interpretacja to piątka uporządkowana zawierająca:
<br>
- formę tekstową,
- lemat (formę bazową/hasłową),
- znacznik morfoskładniowy,
- listę informacji o „pospolitości” rzeczownika (np. nazwa pospolita, marka, nazwisko),
- listę kwalifikatorów stylistycznych (np. daw., pot., środ., wulg.) i dziedzinowych (np. bot., zool.). Segmenty nieznane słownikowi otrzymują specjalny znacznik ign oraz lemat równy formie tekstowej.

<br>
Metoda generate zwraca listę interpretacji morfoskładniowych (w postaci piątek, jw.) wszystkich form, dla których podany tekst stanowi formę bazową:
<br>
*morf.generate(u'piec')*
<br>
W odróżnieniu od analyse, generate akceptuje tylko napisy stanowiące pojedyncze słowo (bez spacji), w przeciwnym przypadku zostanie zgłoszony wyjątek."
<br>

In [34]:
morf = morfeusz2.Morfeusz()

for text in (u'Mam psa', u'Lubisz grać w karty?', u'asdasd123'):
    print(text)
    analysis = morf.analyse(text)
    for interpretation in analysis:
        print(interpretation)

Mam psa
(0, 1, ('Mam', 'mama', 'subst:pl:gen:f', ['nazwa_pospolita'], []))
(0, 1, ('Mam', 'mieć', 'fin:sg:pri:imperf', [], []))
(0, 1, ('Mam', 'mamić', 'impt:sg:sec:imperf', [], []))
(1, 2, ('psa', 'pies:s1', 'subst:sg:gen.acc:m2', ['nazwa_pospolita'], []))
(1, 2, ('psa', 'pies:s2', 'subst:sg:gen.acc:m1', ['nazwa_pospolita'], ['pot.']))
Lubisz grać w karty?
(0, 1, ('Lubisz', 'lubić', 'fin:sg:sec:imperf', [], []))
(1, 2, ('grać', 'grać', 'inf:imperf', [], []))
(2, 3, ('w', 'w', 'prep:acc:nwok', [], []))
(2, 3, ('w', 'w', 'prep:loc:nwok', [], []))
(3, 4, ('karty', 'karty', 'subst:pl:nom.acc.voc:n:pt', ['nazwa_pospolita'], []))
(3, 4, ('karty', 'kart', 'subst:pl:nom.acc.voc:m2', ['nazwa_pospolita'], ['pot.']))
(3, 4, ('karty', 'kart', 'subst:pl:nom.acc.voc:m3', ['nazwa_pospolita'], []))
(3, 4, ('karty', 'karta', 'subst:sg:gen:f', ['nazwa_pospolita'], []))
(3, 4, ('karty', 'karta', 'subst:pl:nom.acc.voc:f', ['nazwa_pospolita'], []))
(4, 5, ('?', '?', 'interp', [], []))
asdasd123
(0, 1, ('a